In [1]:
import requests
import json
import os
import pandas as pd
import numpy as np
from pyproj import Proj
from shapely.geometry import shape
import pickle

In [11]:
#headers = {"token": "lcFPkLrHDzTQGMmQXQlEPFAusVJXevFk"}
headers = {"token": "xVEIkLnfHyheHhvoheZSxesUerlyrxGN"}

In [29]:
all_project_stations = pd.read_csv('./data_2016/all_project_stations.csv')
all_project_stations

,datacoverage,elevation,elevationUnit,id,latitude,longitude,maxdate,mindate,name,square_diagonal,query_count,query_long,query_lat,dist_from_center,p_name
0,0.9995,304.8,METERS,GHCND_USW00013969,36.73667,-97.10194,10/16/2018,1/1/1948,"PONCA CITY MUNICIPAL AIRPORT, OK US",2.139864,14,-97.306297,36.572365,0.262218,Thunder Ranch
1,0.9872,299.9,METERS,GHCND_USW00003965,36.16240,-97.08940,10/16/2018,11/1/1996,"STILLWATER REGIONAL AIRPORT, OK US",2.139864,14,-97.306297,36.572365,0.463805,Thunder Ranch
2,0.9675,272.8,METERS,GHCND_USC00348501,36.11750,-97.09500,10/17/2018,1893-01-01,"STILLWATER 2 W, OK US",2.139864,14,-97.306297,36.572365,0.501546,Thunder Ranch
3,0.8607,1309.4,METERS,GHCND_USW00024030,41.09930,-102.98630,10/16/2018,1/1/1948,"SIDNEY MUNICIPAL AIRPORT, NE US",1.783220,13,-103.322228,40.961955,0.362920,Peetz Table
4,0.9565,1316.7,METERS,GHCND_USC00257830,41.22940,-103.02140,9/30/2018,12/1/1908,"SIDNEY 6 NNW, NE US",1.783220,13,-103.322228,40.961955,0.402522,Peetz Table
5,0.9988,1130.5,METERS,GHCND_USC00051996,40.86000,-102.80300,10/17/2018,3/1/1996,"CROOK, CO US",1.783220,13,-103.322228,40.961955,0.529143,Peetz Table
6,0.9804,545.6,METERS,GHCND_USW00013962,32.41050,-99.68220,10/16/2018,8/1/1946,"ABILENE REGIONAL AIRPORT, TX US",2.567837,15,-100.173964,32.253115,0.516336,Horse Hollow
7,0.9411,594.1,METERS,GHCND_USC00414278,31.84555,-99.56068,10/17/2018,7/1/1953,"HORDS CREEK DAM, TX US",2.567837,15,-100.173964,32.253115,0.736361,Horse Hollow
8,0.8222,598.6,METERS,GHCND_USC00416499,31.45389,-100.49333,1/31/2008,12/1/1975,"O C FISHER DAM, TX US",2.567837,15,-100.173964,32.253115,0.860671,Horse Hollow
9,0.8793,786.4,METERS,GHCND_USW00024012,46.79940,-102.79720,10/16/2018,11/14/1938,"DICKINSON THEODORE ROOSEVELT REGIONAL AIRPORT,...",3.081404,16,-102.756389,46.656348,0.148759,Brady Wind I


In [13]:
len(all_project_stations)

1248

In [14]:
url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/datatypes?datacategoryid=WIND&limit=56"
response = requests.get(url=url, headers=headers)
noaa_wind_data_types_json = response.json()
noaa_wind_data_types_df = pd.DataFrame(noaa_wind_data_types_json['results'])
wind_data_types = noaa_wind_data_types_df.id

In [15]:
noaa_wind_data_types_df

,datacoverage,id,maxdate,mindate,name
0,1,AWND,2018-10-22,1982-01-01,Average wind speed
1,1,DAWM,2010-06-21,1935-09-23,Number of days included in the multiday wind m...
2,1,FMTM,2013-03-31,1982-01-01,Time of fastest mile or fastest 1-minute wind
3,1,HLY-WIND-1STDIR,2010-12-31,2010-01-01,Prevailing wind direction (1-8)
4,1,HLY-WIND-1STPCT,2010-12-31,2010-01-01,Prevailing wind percentage
5,1,HLY-WIND-2NDDIR,2010-12-31,2010-01-01,Secondary wind direction (1-8)
6,1,HLY-WIND-2NDPCT,2010-12-31,2010-01-01,Secondary wind percentage
7,1,HLY-WIND-AVGSPD,2010-12-31,2010-01-01,Average wind speed
8,1,HLY-WIND-PCTCLM,2010-12-31,2010-01-01,Percentage calm
9,1,HLY-WIND-VCTDIR,2010-12-31,2010-01-01,Mean wind vector direction


In [16]:
#parameters for weather query
datasetid='GHCND'
datatypeid="AWND"
stationid='GHCND:USW00013969'
units='standard'
startdate='2016-01-01'
enddate='2016-12-31'
limit = 366

In [17]:
def get_wind_data_daily_summary(stationid, datatypeid, units = 'standard', startdate = '2016-01-01', enddate='2016-12-31', limit =366, offset = 25):
    url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=" +\
    datatypeid + "&stationid=" + stationid + "&units=" + units + "&startdate=" + startdate +\
    "&enddate=" + enddate + "&limit=" + str(limit)
    response = requests.get(url=url, headers=headers)
    station_weather_report_json = response.json()
    station_weather_report_df = pd.DataFrame(station_weather_report_json['results'])
    return(station_weather_report_df)

In [18]:
stations1 = all_project_stations[:350]
stations1

,datacoverage,elevation,elevationUnit,id,latitude,longitude,maxdate,mindate,name,square_diagonal,query_count,query_long,query_lat,dist_from_center,p_name
0,0.9995,304.8,METERS,GHCND_USW00013969,36.73667,-97.10194,10/16/2018,1/1/1948,"PONCA CITY MUNICIPAL AIRPORT, OK US",2.139864,14,-97.306297,36.572365,0.262218,Thunder Ranch
1,0.9872,299.9,METERS,GHCND_USW00003965,36.16240,-97.08940,10/16/2018,11/1/1996,"STILLWATER REGIONAL AIRPORT, OK US",2.139864,14,-97.306297,36.572365,0.463805,Thunder Ranch
2,0.9675,272.8,METERS,GHCND_USC00348501,36.11750,-97.09500,10/17/2018,1893-01-01,"STILLWATER 2 W, OK US",2.139864,14,-97.306297,36.572365,0.501546,Thunder Ranch
3,0.8607,1309.4,METERS,GHCND_USW00024030,41.09930,-102.98630,10/16/2018,1/1/1948,"SIDNEY MUNICIPAL AIRPORT, NE US",1.783220,13,-103.322228,40.961955,0.362920,Peetz Table
4,0.9565,1316.7,METERS,GHCND_USC00257830,41.22940,-103.02140,9/30/2018,12/1/1908,"SIDNEY 6 NNW, NE US",1.783220,13,-103.322228,40.961955,0.402522,Peetz Table
5,0.9988,1130.5,METERS,GHCND_USC00051996,40.86000,-102.80300,10/17/2018,3/1/1996,"CROOK, CO US",1.783220,13,-103.322228,40.961955,0.529143,Peetz Table
6,0.9804,545.6,METERS,GHCND_USW00013962,32.41050,-99.68220,10/16/2018,8/1/1946,"ABILENE REGIONAL AIRPORT, TX US",2.567837,15,-100.173964,32.253115,0.516336,Horse Hollow
7,0.9411,594.1,METERS,GHCND_USC00414278,31.84555,-99.56068,10/17/2018,7/1/1953,"HORDS CREEK DAM, TX US",2.567837,15,-100.173964,32.253115,0.736361,Horse Hollow
8,0.8222,598.6,METERS,GHCND_USC00416499,31.45389,-100.49333,1/31/2008,12/1/1975,"O C FISHER DAM, TX US",2.567837,15,-100.173964,32.253115,0.860671,Horse Hollow
9,0.8793,786.4,METERS,GHCND_USW00024012,46.79940,-102.79720,10/16/2018,11/14/1938,"DICKINSON THEODORE ROOSEVELT REGIONAL AIRPORT,...",3.081404,16,-102.756389,46.656348,0.148759,Brady Wind I


In [22]:
stations2 = all_project_stations[350:700]
stations2

,datacoverage,elevation,elevationUnit,id,latitude,longitude,maxdate,mindate,name,square_diagonal,query_count,query_long,query_lat,dist_from_center,p_name
350,0.9837,97.5,METERS,GHCND_USC00413060,26.55800,-99.13720,10/17/2018,7/1/1962,"FALCON DAM, TX US",2.567837,15,-98.917783,27.459435,0.927755,Javelina II
351,0.8607,1309.4,METERS,GHCND_USW00024030,41.09930,-102.98630,10/16/2018,1/1/1948,"SIDNEY MUNICIPAL AIRPORT, NE US",2.139864,14,-103.019961,40.963056,0.140341,Spring Canyon
352,0.9988,1130.5,METERS,GHCND_USC00051996,40.86000,-102.80300,10/17/2018,3/1/1996,"CROOK, CO US",2.139864,14,-103.019961,40.963056,0.240192,Spring Canyon
353,0.9565,1316.7,METERS,GHCND_USC00257830,41.22940,-103.02140,9/30/2018,12/1/1908,"SIDNEY 6 NNW, NE US",2.139864,14,-103.019961,40.963056,0.266348,Spring Canyon
354,0.9637,179.8,METERS,GHCND_USW00094898,44.02194,-82.79306,10/15/2018,6/24/1999,"PORT HOPE, MI US",2.567837,15,-83.178235,43.891077,0.406799,Echo
355,0.9098,178.6,METERS,GHCND_USC00202423,44.28380,-83.50360,10/17/2018,1890-04-01,"EAST TAWAS, MI US",2.567837,15,-83.178235,43.891077,0.509994,Echo
356,0.8035,256.6,METERS,GHCND_USC00203527,44.31270,-83.82720,9/30/2009,2/1/2002,"HALE 5 SSW, MI US",2.567837,15,-83.178235,43.891077,0.773900,Echo
357,0.9964,694.9,METERS,GHCND_USC00414967,33.06417,-101.04889,10/17/2018,5/1/1994,"LAKE ALAN HENRY, TX US",2.567837,15,-100.988649,32.892499,0.181934,Amazon Wind Farm Texas
358,0.7781,700.1,METERS,GHCND_USC00418566,33.47917,-100.87611,10/16/2018,5/1/1911,"SPUR, TX US",2.567837,15,-100.988649,32.892499,0.597367,Amazon Wind Farm Texas
359,0.9252,765.0,METERS,GHCND_USW00023041,32.24430,-101.45370,10/17/2018,1/1/1948,"BIG SPRING, TX US",2.567837,15,-100.988649,32.892499,0.797769,Amazon Wind Farm Texas


In [28]:
stations3 = all_project_stations[700:1050]
stations3

,datacoverage,elevation,elevationUnit,id,latitude,longitude,maxdate,mindate,name,square_diagonal,query_count,query_long,query_lat,dist_from_center,p_name
700,1.0000,472.7,METERS,GHCND_USW00014941,41.98550,-97.43520,10/16/2018,10/1/1945,"NORFOLK KARL STEFAN MEMORIAL AIRPORT, NE US",3.081404,16,-98.022160,41.879491,0.596456,Laredo Ridge
701,0.9788,515.1,METERS,GHCND_USC00256396,42.36910,-97.59830,7/31/2013,9/1/2008,"OSMOND NUMBER 1, NE US",3.081404,16,-98.022160,41.879491,0.647591,Laredo Ridge
702,0.9436,1442.6,METERS,GHCND_USW00024145,43.51900,-112.06390,10/16/2018,8/1/1948,"IDAHO FALLS FANNING FIELD, ID US",3.081404,16,-111.843263,43.384136,0.258591,Wolverine Creek
703,0.9214,1525.5,METERS,GHCND_USC00107644,43.80830,-111.78920,9/13/2018,7/1/1977,"REXBURG BYU IDAHO, ID US",3.081404,16,-111.843263,43.384136,0.427596,Wolverine Creek
704,0.9404,1481.9,METERS,GHCND_USW00094194,43.83212,-111.80753,10/16/2018,2/12/1998,"REXBURG MADISON CO AIRPORT, ID US",3.081404,16,-111.843263,43.384136,0.449407,Wolverine Creek
705,0.8911,1620.9,METERS,GHCND_USW00024089,42.89779,-106.47371,10/16/2018,1/1/1940,"CASPER NATRONA CO AIRPORT, WY US",3.697685,17,-106.000878,43.014919,0.487123,Campbell Hill
706,0.9990,1504.5,METERS,GHCND_USW00094057,42.79611,-105.38028,10/16/2018,7/1/1999,"CONVERSE CO AIRPORT ASOS, WY US",3.697685,17,-106.000878,43.014919,0.658042,Campbell Hill
707,0.8080,1805.3,METERS,GHCND_USC00487105,42.47165,-106.85268,8/31/2018,10/1/1905,"PATHFINDER DAM, WY US",3.697685,17,-106.000878,43.014919,1.010301,Campbell Hill
708,0.9988,1130.5,METERS,GHCND_USC00051996,40.86000,-102.80300,10/17/2018,3/1/1996,"CROOK, CO US",1.783220,13,-102.752407,40.743612,0.126908,Colorado Highlands (2012)
709,0.9790,1297.2,METERS,GHCND_USC00052947,40.64830,-102.85940,9/30/2018,4/1/1998,"FLEMING 3 SW, CO US",1.783220,13,-102.752407,40.743612,0.143290,Colorado Highlands (2012)


In [31]:
stations4 = all_project_stations[1050:1248]
stations4

,datacoverage,elevation,elevationUnit,id,latitude,longitude,maxdate,mindate,name,square_diagonal,query_count,query_long,query_lat,dist_from_center,p_name
1050,0.3159,18.0,METERS,GHCND_USW00023202,38.26667,-121.93333,10/12/2018,5/24/1947,"TRAVIS FIELD AFB, CA US",0.859963,9,-122.090683,38.232900,0.160936,Anheuser-Busch Fairfield
1051,0.9996,4.3,METERS,GHCND_USW00093227,38.21020,-122.28470,10/16/2018,5/22/1998,"NAPA CO AIRPORT, CA US",0.859963,9,-122.090683,38.232900,0.195340,Anheuser-Busch Fairfield
1052,0.9879,33.2,METERS,GHCND_USW00093241,38.37750,-121.95750,10/16/2018,4/6/1998,"VACAVILLE NUT TREE AIRPORT, CA US",0.859963,9,-122.090683,38.232900,0.196588,Anheuser-Busch Fairfield
1053,0.7574,195.1,METERS,GHCND_USW00054768,42.69728,-73.16967,10/16/2018,10/1/1987,"NORTH ADAMS HARRIMAN AIRPORT, MA US",1.238347,11,-73.274979,42.585271,0.153740,Berkshire Wind
1054,0.6509,363.9,METERS,GHCND_USW00014763,42.42722,-73.28917,10/16/2018,6/1/1925,"PITTSFIELD MUNICIPAL AIRPORT, MA US",1.238347,11,-73.274979,42.585271,0.158687,Berkshire Wind
1055,0.8409,338.3,METERS,GHCND_USC00436500,42.79167,-73.22278,7/20/2013,1/11/1975,"POWNAL 1 NE, VT US",1.238347,11,-73.274979,42.585271,0.212897,Berkshire Wind
1056,1.0000,1116.8,METERS,GHCND_USW00024143,47.47330,-111.38220,10/16/2018,4/1/1937,"GREAT FALLS AIRPORT, MT US",2.567837,15,-111.437870,47.500509,0.061964,Horseshoe Bend&ranch pit
1057,0.9928,1143.0,METERS,GHCND_USC00246700,47.65440,-111.59800,5/31/2014,2/13/1953,"POWER 6 SE, MT US",2.567837,15,-111.437870,47.500509,0.222090,Horseshoe Bend&ranch pit
1058,0.4638,1058.3,METERS,GHCND_USW00024112,47.51667,-111.18333,6/8/2014,1/1/1949,"MALMSTROM AFB, MT US",2.567837,15,-111.437870,47.500509,0.255053,Horseshoe Bend&ranch pit
1059,0.3824,39.6,METERS,GHCND_USW00014704,41.65000,-70.51667,7/30/2010,1/1/1949,"FALMOUTH OTIS AFB, MA US",1.238347,11,-70.546550,41.758591,0.112627,AFCEE MMR Turbines


In [32]:
station_count = 0
query_count = 0
start_date = "2016-01-01"
end_date = "2016-12-31"
for index, row in stations4.iterrows():
    stationid = row['id'].replace("_", ":")
    stationid_dir_name = row["id"]
    projectid = row["p_name"].replace("/","&")
    failed_datatypes = []
    station_count += 1
    for w in wind_data_types:
        try:
            query_count += 1
            print("Project:", projectid, " | Station:", stationid, " | Feature:", w,
                  " | Station Count:", station_count, " | Query Count:", query_count)
            downloaded_data = get_wind_data_daily_summary(stationid, w, units = 'standard',
                                                          startdate=start_date , enddate=end_date, limit=1000)
            
            outname = w + "_" + start_date + "_" + end_date + ".csv"
            outdir = './data_2016/weather_2016/projects/' + projectid + "/" + stationid_dir_name
            if not os.path.exists(outdir):
                os.makedirs(outdir)
            fullname = os.path.join(outdir, outname)    
            downloaded_data.to_csv(fullname)
            
            #file_path = './data/weather_2016/projects/' + projectid + "/" + stationid_dir_name + '/' + w + \
            #"_" + start_date + "_" + end_date + ".csv"
            downloaded_data.to_csv(file_path, index=False)
        except:
            failed_datatypes.append(w)
    
    outname = 'failed_datatypes'
    if not os.path.exists(outdir):
        os.makedirs(outdir)
    fullname = os.path.join(outdir, outname) 
    with open(fullname , 'w') as f:
        for item in failed_datatypes:
            f.write("%s\n" % item)

Project: Anheuser-Busch Fairfield  | Station: GHCND:USW00023202  | Feature: AWND  | Station Count: 1  | Query Count: 1
Project: Anheuser-Busch Fairfield  | Station: GHCND:USW00023202  | Feature: DAWM  | Station Count: 1  | Query Count: 2
Project: Anheuser-Busch Fairfield  | Station: GHCND:USW00023202  | Feature: FMTM  | Station Count: 1  | Query Count: 3
Project: Anheuser-Busch Fairfield  | Station: GHCND:USW00023202  | Feature: HLY-WIND-1STDIR  | Station Count: 1  | Query Count: 4
Project: Anheuser-Busch Fairfield  | Station: GHCND:USW00023202  | Feature: HLY-WIND-1STPCT  | Station Count: 1  | Query Count: 5
Project: Anheuser-Busch Fairfield  | Station: GHCND:USW00023202  | Feature: HLY-WIND-2NDDIR  | Station Count: 1  | Query Count: 6
Project: Anheuser-Busch Fairfield  | Station: GHCND:USW00023202  | Feature: HLY-WIND-2NDPCT  | Station Count: 1  | Query Count: 7
Project: Anheuser-Busch Fairfield  | Station: GHCND:USW00023202  | Feature: HLY-WIND-AVGSPD  | Station Count: 1  | Query Co

Project: Anheuser-Busch Fairfield  | Station: GHCND:USW00093241  | Feature: TWND  | Station Count: 3  | Query Count: 68
Project: Anheuser-Busch Fairfield  | Station: GHCND:USW00093241  | Feature: WDF1  | Station Count: 3  | Query Count: 69
Project: Anheuser-Busch Fairfield  | Station: GHCND:USW00093241  | Feature: WDF2  | Station Count: 3  | Query Count: 70
Project: Anheuser-Busch Fairfield  | Station: GHCND:USW00093241  | Feature: WDF5  | Station Count: 3  | Query Count: 71
Project: Anheuser-Busch Fairfield  | Station: GHCND:USW00093241  | Feature: WDFG  | Station Count: 3  | Query Count: 72
Project: Anheuser-Busch Fairfield  | Station: GHCND:USW00093241  | Feature: WDFI  | Station Count: 3  | Query Count: 73
Project: Anheuser-Busch Fairfield  | Station: GHCND:USW00093241  | Feature: WDFM  | Station Count: 3  | Query Count: 74
Project: Anheuser-Busch Fairfield  | Station: GHCND:USW00093241  | Feature: WDMV  | Station Count: 3  | Query Count: 75
Project: Anheuser-Busch Fairfield  | Sta

Project: Berkshire Wind  | Station: GHCND:USC00436500  | Feature: HLY-WIND-2NDDIR  | Station Count: 6  | Query Count: 141
Project: Berkshire Wind  | Station: GHCND:USC00436500  | Feature: HLY-WIND-2NDPCT  | Station Count: 6  | Query Count: 142
Project: Berkshire Wind  | Station: GHCND:USC00436500  | Feature: HLY-WIND-AVGSPD  | Station Count: 6  | Query Count: 143
Project: Berkshire Wind  | Station: GHCND:USC00436500  | Feature: HLY-WIND-PCTCLM  | Station Count: 6  | Query Count: 144
Project: Berkshire Wind  | Station: GHCND:USC00436500  | Feature: HLY-WIND-VCTDIR  | Station Count: 6  | Query Count: 145
Project: Berkshire Wind  | Station: GHCND:USC00436500  | Feature: HLY-WIND-VCTSPD  | Station Count: 6  | Query Count: 146
Project: Berkshire Wind  | Station: GHCND:USC00436500  | Feature: MDWM  | Station Count: 6  | Query Count: 147
Project: Berkshire Wind  | Station: GHCND:USC00436500  | Feature: PGTM  | Station Count: 6  | Query Count: 148
Project: Berkshire Wind  | Station: GHCND:USC0

Project: Horseshoe Bend&ranch pit  | Station: GHCND:USC00246700  | Feature: WDFM  | Station Count: 8  | Query Count: 209
Project: Horseshoe Bend&ranch pit  | Station: GHCND:USC00246700  | Feature: WDMV  | Station Count: 8  | Query Count: 210
Project: Horseshoe Bend&ranch pit  | Station: GHCND:USC00246700  | Feature: WSF1  | Station Count: 8  | Query Count: 211
Project: Horseshoe Bend&ranch pit  | Station: GHCND:USC00246700  | Feature: WSF2  | Station Count: 8  | Query Count: 212
Project: Horseshoe Bend&ranch pit  | Station: GHCND:USC00246700  | Feature: WSF5  | Station Count: 8  | Query Count: 213
Project: Horseshoe Bend&ranch pit  | Station: GHCND:USC00246700  | Feature: WSFG  | Station Count: 8  | Query Count: 214
Project: Horseshoe Bend&ranch pit  | Station: GHCND:USC00246700  | Feature: WSFI  | Station Count: 8  | Query Count: 215
Project: Horseshoe Bend&ranch pit  | Station: GHCND:USC00246700  | Feature: WSFM  | Station Count: 8  | Query Count: 216
Project: Horseshoe Bend&ranch pi

Project: AFCEE MMR Turbines  | Station: GHCND:USW00054769  | Feature: HLY-WIND-2NDPCT  | Station Count: 11  | Query Count: 277
Project: AFCEE MMR Turbines  | Station: GHCND:USW00054769  | Feature: HLY-WIND-AVGSPD  | Station Count: 11  | Query Count: 278
Project: AFCEE MMR Turbines  | Station: GHCND:USW00054769  | Feature: HLY-WIND-PCTCLM  | Station Count: 11  | Query Count: 279
Project: AFCEE MMR Turbines  | Station: GHCND:USW00054769  | Feature: HLY-WIND-VCTDIR  | Station Count: 11  | Query Count: 280
Project: AFCEE MMR Turbines  | Station: GHCND:USW00054769  | Feature: HLY-WIND-VCTSPD  | Station Count: 11  | Query Count: 281
Project: AFCEE MMR Turbines  | Station: GHCND:USW00054769  | Feature: MDWM  | Station Count: 11  | Query Count: 282
Project: AFCEE MMR Turbines  | Station: GHCND:USW00054769  | Feature: PGTM  | Station Count: 11  | Query Count: 283
Project: AFCEE MMR Turbines  | Station: GHCND:USW00054769  | Feature: TWND  | Station Count: 11  | Query Count: 284
Project: AFCEE MM

...

Project: Cumberland Rose  | Station: GHCND:USC00133438  | Feature: WSF2  | Station Count: 187  | Query Count: 5045
Project: Cumberland Rose  | Station: GHCND:USC00133438  | Feature: WSF5  | Station Count: 187  | Query Count: 5046
Project: Cumberland Rose  | Station: GHCND:USC00133438  | Feature: WSFG  | Station Count: 187  | Query Count: 5047
Project: Cumberland Rose  | Station: GHCND:USC00133438  | Feature: WSFI  | Station Count: 187  | Query Count: 5048
Project: Cumberland Rose  | Station: GHCND:USC00133438  | Feature: WSFM  | Station Count: 187  | Query Count: 5049
Project: Cumberland Rose  | Station: GHCND:USW00094991  | Feature: AWND  | Station Count: 188  | Query Count: 5050
Project: Cumberland Rose  | Station: GHCND:USW00094991  | Feature: DAWM  | Station Count: 188  | Query Count: 5051
Project: Cumberland Rose  | Station: GHCND:USW00094991  | Feature: FMTM  | Station Count: 188  | Query Count: 5052
Project: Cumberland Rose  | Station: GHCND:USW00094991  | Feature: HLY-WIND-1STD

Project: Crow Lake  | Station: GHCND:USW00094943  | Feature: MDWM  | Station Count: 190  | Query Count: 5115
Project: Crow Lake  | Station: GHCND:USW00094943  | Feature: PGTM  | Station Count: 190  | Query Count: 5116
Project: Crow Lake  | Station: GHCND:USW00094943  | Feature: TWND  | Station Count: 190  | Query Count: 5117
Project: Crow Lake  | Station: GHCND:USW00094943  | Feature: WDF1  | Station Count: 190  | Query Count: 5118
Project: Crow Lake  | Station: GHCND:USW00094943  | Feature: WDF2  | Station Count: 190  | Query Count: 5119
Project: Crow Lake  | Station: GHCND:USW00094943  | Feature: WDF5  | Station Count: 190  | Query Count: 5120
Project: Crow Lake  | Station: GHCND:USW00094943  | Feature: WDFG  | Station Count: 190  | Query Count: 5121
Project: Crow Lake  | Station: GHCND:USW00094943  | Feature: WDFI  | Station Count: 190  | Query Count: 5122
Project: Crow Lake  | Station: GHCND:USW00094943  | Feature: WDFM  | Station Count: 190  | Query Count: 5123
Project: Crow Lake 

Project: City of Soledad Water Reclamation Facility  | Station: GHCND:USW00023233  | Feature: HLY-WIND-1STDIR  | Station Count: 193  | Query Count: 5188
Project: City of Soledad Water Reclamation Facility  | Station: GHCND:USW00023233  | Feature: HLY-WIND-1STPCT  | Station Count: 193  | Query Count: 5189
Project: City of Soledad Water Reclamation Facility  | Station: GHCND:USW00023233  | Feature: HLY-WIND-2NDDIR  | Station Count: 193  | Query Count: 5190
Project: City of Soledad Water Reclamation Facility  | Station: GHCND:USW00023233  | Feature: HLY-WIND-2NDPCT  | Station Count: 193  | Query Count: 5191
Project: City of Soledad Water Reclamation Facility  | Station: GHCND:USW00023233  | Feature: HLY-WIND-AVGSPD  | Station Count: 193  | Query Count: 5192
Project: City of Soledad Water Reclamation Facility  | Station: GHCND:USW00023233  | Feature: HLY-WIND-PCTCLM  | Station Count: 193  | Query Count: 5193
Project: City of Soledad Water Reclamation Facility  | Station: GHCND:USW00023233 

Project: City of Soledad Water Reclamation Facility  | Station: GHCND:USW00023277  | Feature: HLY-WIND-2NDPCT  | Station Count: 195  | Query Count: 5245
Project: City of Soledad Water Reclamation Facility  | Station: GHCND:USW00023277  | Feature: HLY-WIND-AVGSPD  | Station Count: 195  | Query Count: 5246
Project: City of Soledad Water Reclamation Facility  | Station: GHCND:USW00023277  | Feature: HLY-WIND-PCTCLM  | Station Count: 195  | Query Count: 5247
Project: City of Soledad Water Reclamation Facility  | Station: GHCND:USW00023277  | Feature: HLY-WIND-VCTDIR  | Station Count: 195  | Query Count: 5248
Project: City of Soledad Water Reclamation Facility  | Station: GHCND:USW00023277  | Feature: HLY-WIND-VCTSPD  | Station Count: 195  | Query Count: 5249
Project: City of Soledad Water Reclamation Facility  | Station: GHCND:USW00023277  | Feature: MDWM  | Station Count: 195  | Query Count: 5250
Project: City of Soledad Water Reclamation Facility  | Station: GHCND:USW00023277  | Feature:

Project: Sky Volt  | Station: GHCND:USW00014933  | Feature: WDMV  | Station Count: 197  | Query Count: 5313
Project: Sky Volt  | Station: GHCND:USW00014933  | Feature: WSF1  | Station Count: 197  | Query Count: 5314
Project: Sky Volt  | Station: GHCND:USW00014933  | Feature: WSF2  | Station Count: 197  | Query Count: 5315
Project: Sky Volt  | Station: GHCND:USW00014933  | Feature: WSF5  | Station Count: 197  | Query Count: 5316
Project: Sky Volt  | Station: GHCND:USW00014933  | Feature: WSFG  | Station Count: 197  | Query Count: 5317
Project: Sky Volt  | Station: GHCND:USW00014933  | Feature: WSFI  | Station Count: 197  | Query Count: 5318
Project: Sky Volt  | Station: GHCND:USW00014933  | Feature: WSFM  | Station Count: 197  | Query Count: 5319
Project: Sky Volt  | Station: GHCND:USW00094991  | Feature: AWND  | Station Count: 198  | Query Count: 5320
Project: Sky Volt  | Station: GHCND:USW00094991  | Feature: DAWM  | Station Count: 198  | Query Count: 5321
Project: Sky Volt  | Station